In [1]:
import pyodbc
print(pyodbc.drivers())
conn_str = (
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=157.119.230.120,4071;"
    "Database=PowerBI_LISV2;"
    "UID=sa;"
    "PWD=Y0m@Sql.!.123;"
)

['SQL Server', 'SQL Server Native Client 11.0', 'ODBC Driver 11 for SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'ODBC Driver 17 for SQL Server']


In [2]:
import pyodbc
import pandas as pd
conn = pyodbc.connect(conn_str)

In [3]:
final_df =  pd.DataFrame()
x = ['AmulManagement', 'BrillonMTManagement','EvereadyManagement','IDAMManagement','MTRManagement','PidiliteManagement',
     'PiramalManagement','ReckittManagement','WiproGranammaManagement','ZydusMTManagement','HimalayaMTManagement','DanoneMTManagement',
     'GCPLManagement','JohnsonJohnsonManagement','ParleManagement',]

In [6]:
for value in x:
    query = f"""
        SELECT distinct '{value}' as Project,State, City,[LISStoreCode],[StoreName],[ChainName],[InLatitude],[InLongitude]
        FROM [PowerBI_LISV2].[{value}].[ISPStoreAttendance_NormalExport]
        WHERE ([LISStoreCode] NOT LIKE '%Test%' OR [LISStoreCode] NOT LIKE '%test%');
    """

    df = pd.read_sql(query, conn)
    final_df = pd.concat([final_df, df], ignore_index=True)
    print(f"Running for: {value}")

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_20360\3593469400.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Running for: AmulManagement
Running for: BrillonMTManagement
Running for: EvereadyManagement
Running for: IDAMManagement
Running for: MTRManagement
Running for: PidiliteManagement
Running for: PiramalManagement
Running for: ReckittManagement
Running for: WiproGranammaManagement
Running for: ZydusMTManagement
Running for: HimalayaMTManagement


C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_20360\3593469400.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Running for: DanoneMTManagement
Running for: GCPLManagement
Running for: JohnsonJohnsonManagement
Running for: ParleManagement


In [6]:
# Query
query = """SELECT distinct State, City,[LISStoreCode],[StoreName],[ChainName],[InLatitude],[InLongitude]
FROM [PowerBI_LISV2].[PidiliteManagement].[ISPStoreAttendance_NormalExport]
WHERE ([LISStoreCode] NOT LIKE '%Test%' OR [LISStoreCode] NOT LIKE '%test%');"""
# df = pd.read_sql(query, conn)

C:\Users\kunal\AppData\Local\Temp\ipykernel_10308\94442313.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [11]:
final_df.drop_duplicates()
final_df.head()

Project      State        City    LISStoreCode  StoreName  \
0         AmulManagement      Assam    Guwahati   AMLStore-1253  Best Pick   
1         AmulManagement      Assam    Guwahati   AMLStore-1253  Best Pick   
2         AmulManagement      Assam    Guwahati   AMLStore-1253  Best Pick   
3         AmulManagement      Assam    Guwahati   AMLStore-1253  Best Pick   
4         AmulManagement      Assam    Guwahati   AMLStore-1253  Best Pick   
...                  ...        ...         ...             ...        ...   
2130071  ParleManagement  Telangana  Zaheerabad  Parle-Store-49   SRR MORE   
2130072  ParleManagement  Telangana  Zaheerabad  Parle-Store-49   SRR MORE   
2130073  ParleManagement  Telangana  Zaheerabad  Parle-Store-49   SRR MORE   
2130074  ParleManagement  Telangana  Zaheerabad  Parle-Store-49   SRR MORE   
2130075  ParleManagement  Telangana  Zaheerabad  Parle-Store-49   SRR MORE   

         ChainName  InLatitude InLongitude  
0        Local MFS  26.1333784  91.8026163  
1        Local MFS   26.133664  91.8030514  
2        Local MFS  26.1336714  91.8029599  
3        Local MFS  26.1336844  91.8029746  
4        Local MFS  26.1336934  91.8029597  
...            ...         ...         ...  
2130071       ABRL  17.6807053  77.6165403  
2130072       ABRL  17.6808683  77.6165909  
2130073       ABRL  17.6809645  77.6166612  
2130074       ABRL  17.6815784  77.6226288  
2130075       ABRL  17.6816734  77.6225776  

[2130076 rows x 8 columns]

In [12]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2130076 entries, 0 to 2130075
Data columns (total 8 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   Project       object
 1   State         object
 2   City          object
 3   LISStoreCode  object
 4   StoreName     object
 5   ChainName     object
 6   InLatitude    object
 7   InLongitude   object
dtypes: object(8)
memory usage: 130.0+ MB


### Clustering

In [13]:
df = final_df

In [17]:
import numpy as np
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

from sklearn.preprocessing import StandardScaler
from math import radians

In [16]:
# !pip install shapely

In [18]:
df['latitude'] = df['InLatitude'].astype(float)
df['longitude'] = df['InLongitude'].astype(float)

In [19]:

# To store results
store_centroids = []

# Group by store and process each separately
for LISStoreCode, group in df.groupby('LISStoreCode'):
    coords = group[['latitude', 'longitude']].to_numpy()
    coords_rad = np.radians(coords)

    # DBSCAN parameters
    kms_per_radian = 6371.0088
    epsilon = 0.8 / kms_per_radian  # 800 meters
    db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(coords_rad)

    group = group.copy()
    group['cluster'] = db.labels_

    # Filter out noise
    clustered = group[group['cluster'] != -1]

    if clustered.empty:
        # print(f"No valid cluster for store {LISStoreCode}")
        continue

    # Pick largest cluster
    main_cluster = clustered['cluster'].value_counts().idxmax()
    main_cluster_points = clustered[clustered['cluster'] == main_cluster]

    # Compute centroid of the main cluster
    centroid_lat = main_cluster_points['latitude'].mean()
    centroid_lon = main_cluster_points['longitude'].mean()

    store_centroids.append({
        'store_id': LISStoreCode,
        'centroid_latitude': centroid_lat,
        'centroid_longitude': centroid_lon
    })



In [20]:
# Final result
centroid_df = pd.DataFrame(store_centroids)

In [31]:
centroid_df[27000:].head(20)

store_id  centroid_latitude  centroid_longitude
27000  Parle-Store-1327          17.485888           78.490109
27001  Parle-Store-1328          18.441348           79.095202
27002  Parle-Store-1329          17.522962           78.374589
27003   Parle-Store-133          19.782088           72.789098
27004  Parle-Store-1330          17.138938           79.626209
27005  Parle-Store-1331          17.387239           78.512826
27006  Parle-Store-1332          17.604550           78.083196
27007  Parle-Store-1333          17.467116           78.309396
27008  Parle-Store-1334          17.438247           78.395062
27009  Parle-Store-1335          17.372533           78.546373
27010  Parle-Store-1336          17.391434           78.441649
27011  Parle-Store-1337          17.509192           78.505628
27012  Parle-Store-1338          17.437184           78.665321
27013  Parle-Store-1339          17.430414           78.553256
27014   Parle-Store-134          14.672934           77.580629
27015  Parle-Store-1340          17.336092           78.467550
27016  Parle-Store-1341          18.664775           78.109379
27017  Parle-Store-1342          16.865526           79.558559
27018  Parle-Store-1343          17.534318           78.252186
27019  Parle-Store-1344          19.072299           78.349871

In [32]:
centroid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40921 entries, 0 to 40920
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   store_id            40921 non-null  object 
 1   centroid_latitude   40921 non-null  float64
 2   centroid_longitude  40921 non-null  float64
dtypes: float64(2), object(1)
memory usage: 959.2+ KB


In [33]:
centroid_df.to_csv('centroid_df.csv')

In [35]:
# Convert lat-long to NumPy array
coords = centroid_df[['centroid_latitude', 'centroid_longitude']].to_numpy()

In [36]:
# Standardize coordinates
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(coords)

In [37]:
db = DBSCAN(eps=0.000197, min_samples=5, metric='haversine').fit(coords_scaled)

In [38]:
centroid_df['cluster_id'] = db.labels_

In [39]:
centroid_df.head()

store_id  centroid_latitude  centroid_longitude  cluster_id
0   ABRL033          12.879980           77.646227           0
1  ABRLH002          12.310940           76.648061          -1
2  ABRLH006          12.952233           77.567472           1
3  ABRLH009          12.989242           77.688960           2
4  ABRLH014          28.658243           77.144483           3

In [40]:
centroid_df.to_csv('centroid_df.csv')

In [49]:
# coords = df[['latitude', 'longitude']].to_numpy()

In [20]:
# coords_rad = np.radians(coords)

In [22]:
# kms_per_radian = 6371.0088
# epsilon = 0.1 / kms_per_radian  # 100 meters

In [23]:
# db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine').fit(coords_rad)

In [24]:
labels = db.labels_
# df['cluster'] = labels

In [50]:
# group = group.copy()
# group['cluster'] = db.labels_